<div id="body">
    <center>
    <a href="08 Conflicts.ipynb">  <font size="8"> &lt </font></a>
    <a href="index.ipynb">  <font size="8"> Version Control with Git </font> </a>
    <font size="8"> &gt </font>
    </center>
</div>

# **Notes on the use of GIT and GitHub**

## **One time only**

### **Fork**

First ​fork the ORG/repository in the GitHub UI to your_GH_account. This is the same as what GitHub documentation suggests. See: ​[Fork a repo](https://help.github.com/en/articles/fork-a-repo) and [​Syncing a fork](https://help.github.com/en/articles/syncing-a-fork) in GitHub help.

Note: add SSH key, see ​[GitHub documentation](https://help.github.com/en/articles/connecting-to-github-with-ssh).


#### **Origin**

```bash
# "origin" points to your fork repo (e.g. forked from git@github.com:org/repository.git) - IMPORTANT
git clone git@github.com:your_GH_account/repository.git
```

### **Remote**
```bash
# add "upstream" remote
cd repository/
git remote add upstream git@github.com:org/repository.git

git remote -v
# you should see something like
origin	  git@github.com:your_GH_account/repository.git (fetch)
origin	  git@github.com:your_GH_account/repository.git (push)
upstream  git@github.com:org/repository.git (fetch)
upstream  git@github.com:org/repository.git (push)
```

## **Working with git**:



### Editing
```bash
# <make local source code changes>
vim ...
```



### `git fetch`
```bash
# fetch all branches from all remotes
git fetch --all
```



### `git branch`
```bash
# list existing branches
git branch -a
```



### `git checkout`
```bash
# create new local branch (pick a new name for feature_branch_name)
git checkout -b feature_branch_name
```



### `git status`
```bash
# list local changes
git status
```



### `git add`
```bash
# add files and/or changes into the repository
git add file1.c file2.py ...
```


### `git commit`
```bash
# commit
git commit -m 'my change with reasonable explanation...'
```

### `git push`
```bash
# push feature branch to origin, i.e. your fork of the org/repository repo
git push origin feature_branch_name
```

### Pull Request

* create pull request in GitHub Web interface (the link is then shown in the terminal)

* during PR review phase, make more local changes if needed

```bash
git add .
git commit -m 'my second change'
git push origin feature_branch_name
# ..... will be added to existing pull request
```

* NOTE: for different pull requests, simply create different feature branches.


## **Keep your local source code up to date**

[from ​https://github.com/OSGeo/gdal/blob/master/CONTRIBUTING.md#working-with-a-feature-branch]

You may need to resynchronize against master if you need some bugfix or new capability that has been added since you created your branch


### `git rebase`
```bash
# assuming that "upstream" points to org/repository
git fetch upstream
git rebase upstream/master
```


### `git stash`
```bash
# if rebase fails with "error: cannot rebase: You have unstaged changes...", then move your uncommitted local changes to "stash"
git stash
# now you can rebase
git rebase upstream/master
# apply your local changes on top
git stash apply && git stash pop
```

Continue do your changes and commit/push them (ideally to a feature branch, see above).



## Working on private repository

Assuming you are working on your own repository, then direct can be used (instead of forking and PR):

### One time only:

```bash
# "origin" points to your repo - no fork needed
git clone git@github.com:your_GH_account/repository.git
```

### **Work with git:**

```bash
# <make local source code changes>
vim ...

# list local changes
git status
git add file1.c file2.py ...
git commit -m 'my change with reasonable explanation...'

# assuming that "origin" points to your_git_account/repository
git fetch origin
# IMPORTANT - ALWAYS REBASE IN ORDER TO AVOID NOT NEEDED MERGE COMMITS (!!!)
git rebase origin/master

# push feature branch to origin, i.e. directly to your_git_account/repository
git push origin
```

#### **Switching between branches**
For an elegant way of multi-branches in separate directories with only a single repo clone, see

* https://lists.osgeo.org/pipermail/grass-dev/2019-May/092653.html

* Fixing bugs in a release branch

To directly fix bugs (ideally via feature branch), do

```bash
# push to release_branch, we assume it to be checked out

# M_m (Major, minor version number: i.e.: V 1.3)
cd releasebranch_M_m/
# be sure to locally have all updates from server
git fetch --all
git branch --merged 

# create feature branch
git checkout -b rMm_fix_xxx

# ... do changes...

git status
git add ...
git commit -m 'useful commit msg...'

# push to feature branch
git push upstream rMm_fix_xxx

# create PR in GitHub UI. IMPORTANT: switch there to release_branch_X_Y!

# ... after review, merge:

# switch to release branch
git checkout releasebranch_M_m 

# be sure to locally have all updates
git fetch --all
git branch --merged 

git branch -D changelog_fix_msg_Mm 
git fetch --all --prune
git branch -a
```

#### **Backporting to release branches**
* **Preparation**
If you checked out the release branch into a separate directory, be sure to have "upstream" enabled as a remote:

```bash
git remote -v
# if upstream is missing, execute
git remote add upstream git@github.com:org/repository.git
```

#### **Backporting of a single commit**

```bash
git checkout master
# With git log, identify the sha1sum of the commit you want to backport (example: into releasebranch_M_m)
git log

# switch to branch
git checkout releasebranch_M_m

# first update local repo 
git pull origin releasebranch_M_m --rebase

# now backport the commit (edit conflicts if needed)
git cherry-pick the_sha1_sum

# push backport to upstream
git push upstream releasebranch_M_m
```

See also ​https://github.com/OSGeo/gdal/blob/master/CONTRIBUTING.md#backporting-bugfixes-from-master-to-a-stable-branch

#### **Backporting of a merged pull request from master**
Same as "Backporting of single commits" but with multiple git cherry-pick .... Importantly, in the right order.

TODO: there must be a better way!! 

#### **Made a mess? Fix it**

Example: mess happened on releasebranch_M_m:

```bash
git reset --hard upstream/releasebranch_M_m 
git pull upstream releasebranch_M_m --rebase
# now all should be clean again
```


#### **Merging of Pull Requests**

Rationale: We should try to have clean history and good commit messages. This helps really a lot when you try to understand why something is implemented the way it is.

When a Pull Requests (PR) has multiple commits, the merge commit is more or less mandatory because if you don't have it, you can't use `git revert`.

#### **PR with single commit**
Proposed: when a PR only has a single commit, the "merge commit" doesn't offer anything and it can be avoided by rebasing the feature branch:

Workflow: GitHub > button "Merge pull request" > "Rebase and merge"

Next, you may locally delete the feature branch.

#### **PR with multiple commits**

#### **Proposed**: it is a good idea to try to squash the accumulated commits in a PR before merging, especially if those are trivial fixes.

As an example, PRxx contains 5 commits. Esp. in case that several commits of them are trivial fixes that only add noise to the history, "squashing" those results in a cleaner history and, among other things, makes it easier to use `git bisect` and `git blame`.

Importantly, not always commits of each and every PR need to be squashed before merging. When extensive changes are being made, it often makes sense to keep them unsquashed (e.g. to make reviewing easier), but trivial fixes should still be squashed to the main commits.

Further reading
Git Cheatsheet: ​http://ndpsoftware.com/git-cheatsheet.html#loc=workspace; (click on a field to see the related git commands)
GDAL contributing: ​https://github.com/OSGeo/gdal/blob/master/CONTRIBUTING.md


-- Those notes are heavly based on https://trac.osgeo.org/grass/wiki/HowToGit and should be considered as a draft/working-progress starting point on how to make good use of GIT.



<div id="body">
    <center>
    <a href="08 Conflicts.ipynb">  <font size="4"> &lt </font></a>
    <a href="index.ipynb">  <font size="4"> Version Control with Git </font> </a>
    <font size="4"> &gt </font>
    </center>
</div>